[辻真吾・矢吹太朗『ゼロからはじめるデータサイエンス入門』（講談社, 2021）](https://github.com/taroyabuki/fromzero)

In [ ]:
# Google Colaboratoryの環境設定
if (file.exists("/content")) {
  options(Ncpus = parallel::detectCores())
  installed_packages <- rownames(installed.packages())
  packages_to_install <- c("caret", "furrr", "psych", "randomForest", "rpart.plot", "xgboost")
  install.packages(setdiff(packages_to_install, installed_packages))
}

## 9.1 アヤメのデータ

In [ ]:
my_data <- iris
head(my_data)

In [ ]:
psych::describe(my_data)

## 9.2 木による分類

In [ ]:
library(caret)
library(tidyverse)
my_data <- iris
my_model <- train(form = Species ~ ., data = my_data, method = "rpart")

In [ ]:
rpart.plot::rpart.plot(my_model$finalModel, extra = 1)

In [ ]:
my_test <- tribble(
~Sepal.Length, ~Sepal.Width, ~Petal.Length, ~Petal.Width,
          5.0,          3.5,           1.5,          0.5,
          6.5,          3.0,           5.0,          2.0)

my_model %>% predict(my_test)

In [ ]:
my_model %>% predict(my_test,
                     type = "prob")

## 9.3 正解率

In [ ]:
library(caret)
library(tidyverse)
my_data <- iris
my_model <- train(form = Species ~ ., data = my_data, method = "rpart2")

y  <- my_data$Species
y_ <- my_model %>% predict(my_data)
confusionMatrix(data = y_, reference = y)
# 以下は割愛

In [ ]:
y  <- my_data$Species
y_ <- my_model %>% predict(my_data)
mean(y_ == y)

In [ ]:
my_model <- train(form = Species ~ ., data = my_data, method = "rpart2",
                  trControl = trainControl(method = "LOOCV"))
my_model$results

In [ ]:
my_model <- train(form = Species ~ ., data = my_data, method = "rpart2",
                  tuneGrid = data.frame(maxdepth = 1:10),
                  trControl = trainControl(method = "LOOCV"))
my_model$results %>% filter(Accuracy == max(Accuracy))

In [ ]:
# パラメータを与えると正解率（LOOCV）を返す関数
my_loocv <- function(maxdepth, minbucket, minsplit) {
  my_model <- train(form = Species ~ ., data = my_data, method = "rpart2",
                    trControl = trainControl(method = "LOOCV"),
                    tuneGrid = data.frame(maxdepth = maxdepth),
                    control = rpart::rpart.control(cp = 0.01,
                                                   minbucket = minbucket,
                                                   minsplit = minsplit))
  list(maxdepth = maxdepth,
       minbucket = minbucket,
       minsplit = minsplit,
       Accuracy = my_model$results$Accuracy)
}

In [ ]:
my_params <- expand.grid(
  maxdepth = 2:5,
  minbucket = 1:7,
  minsplit = c(2, 20))

library(furrr)
plan(multisession) # 並列処理の準備
my_results <- my_params %>% future_pmap_dfr(my_loocv, # 実行
  .options = furrr_options(seed = TRUE))

my_results %>% filter(Accuracy == max(Accuracy)) # 正解率（検証）の最大値

In [ ]:
my_model <- train(form = Species ~ ., data = my_data, method = "rpart2",
                  trControl = trainControl(method = "none"),
                  tuneGrid = data.frame(maxdepth = 3),
                  control = rpart::rpart.control(cp = 0.01,
                                                 minbucket = 5,
                                                 minsplit = 2))

In [ ]:
rpart.plot::rpart.plot(
  my_model$finalModel, extra = 1)

## 9.4 複数の木を使う方法

In [ ]:
library(caret)
library(tidyverse)
my_data <- iris

my_model <- train(form = Species ~ ., data = my_data, method = "rf",
                  tuneGrid = data.frame(mtry = 2:4), # 省略可
                  trControl = trainControl(method = "LOOCV"))
my_model$results

In [ ]:
my_model <- train(
  form = Species ~ ., data = my_data, method = "xgbTree",
  tuneGrid = expand.grid(
    nrounds          = c(50, 100, 150),
    max_depth        = c(1, 2, 3),
    eta              = c(0.3, 0.4),
    gamma            = 0,
    colsample_bytree = c(0.6, 0.8),
    min_child_weight = 1,
    subsample        = c(0.5, 0.75, 1)),
  trControl = trainControl(method = "cv", number = 5)) # 5分割交差検証
my_model$results %>% filter(Accuracy == max(Accuracy)) %>% head(5) %>% t

In [ ]:
my_model <- train(form = Species ~ ., data = my_data, method = "rf")
ggplot(varImp(my_model))

## 9.5 欠損のあるデータでの学習

In [ ]:
library(caret)
library(tidyverse)
my_data <- iris

n <- nrow(my_data)
my_data$Petal.Length[0:(n - 1) %% 10 == 0] <- NA
my_data$Petal.Width[ 0:(n - 1) %% 10 == 1] <- NA

psych::describe(my_data) # nの値が135の変数に，150 - 135 = 15個の欠損がある．

In [ ]:
my_model <- train(
  form = Species ~ ., data = my_data, method = "rpart2",
  na.action = na.pass,         # 欠損があっても学習を止めない．
  preProcess = "medianImpute", # 欠損を中央値で埋める．
  trControl = trainControl(method = "LOOCV"),
  tuneGrid = data.frame(maxdepth = 20),          # 木の高さの上限
  control = rpart::rpart.control(minsplit = 2,   # 分岐の条件
                                 minbucket = 1)) # 終端ノードの条件
max(my_model$results$Accuracy)

In [ ]:
my_model <- train(form = Species ~ ., data = my_data, method = "xgbTree",
                  na.action = na.pass,
                  trControl = trainControl(method = "cv", number = 5))
max(my_model$results$Accuracy)

## 9.6 他の分類手法

In [ ]:
library(caret)
library(tidyverse)
my_data <- iris

my_model <- train(form = Species ~ ., data = my_data, method = "knn",
                  trControl = trainControl(method = "LOOCV"))
my_model$results %>% filter(Accuracy == max(Accuracy))

In [ ]:
library(caret)
library(tidyverse)
my_data <- iris

my_model <- train(form = Species ~ ., data = my_data, method = "nnet",
                  preProcess = c("center", "scale"), # 標準化
                  trControl = trainControl(method = "LOOCV"),
                  trace = FALSE) # 途中経過を表示しない
my_model$results %>% filter(Accuracy == max(Accuracy))